In [1]:
import pandas as pd
import numpy as np
import textwrap
import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer, PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/kiavashzareei/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/kiavashzareei/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [3]:
# https://www.kaggle.com/shivamkushwaha/bbc-full-text-document-classification
!wget -nc https://lazyprogrammer.me/course_files/nlp/bbc_text_cls.csv

--2023-05-10 18:56:29--  https://lazyprogrammer.me/course_files/nlp/bbc_text_cls.csv
Resolving lazyprogrammer.me (lazyprogrammer.me)... 172.67.213.166, 104.21.23.210, 2606:4700:3031::6815:17d2, ...
Connecting to lazyprogrammer.me (lazyprogrammer.me)|172.67.213.166|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5085081 (4.8M) [text/csv]
Saving to: ‘bbc_text_cls.csv’

bbc_text_cls.csv    100%[===================>]   4.85M  1.31MB/s    in 3.7s    

2023-05-10 18:56:35 (1.31 MB/s) - ‘bbc_text_cls.csv’ saved [5085081/5085081]



In [4]:
df = pd.read_csv('bbc_text_cls.csv')

In [5]:
df.head()

,text,labels
0,Ad sales boost Time Warner profit\n\nQuarterly...,business
1,Dollar gains on Greenspan speech\n\nThe dollar...,business
2,Yukos unit buyer faces loan claim\n\nThe owner...,business
3,High fuel prices hit BA's profits\n\nBritish A...,business
4,Pernod takeover talk lifts Domecq\n\nShares in...,business


In [6]:
doc = df[df.labels =='business']['text'].sample(random_state=42)

In [7]:
def wrap(x):
    return textwrap.fill(x,replace_whitespace=False,fix_sentence_endings = True)

In [8]:
print(wrap(doc.iloc[0]))

Christmas sales worst since 1981

UK retail sales fell in December,
failing to meet expectations and making it by some counts the worst
Christmas since 1981.

Retail sales dropped by 1% on the month in
December, after a 0.6% rise in November, the Office for National
Statistics (ONS) said.  The ONS revised the annual 2004 rate of growth
down from the 5.9% estimated in November to 3.2%. A number of
retailers have already reported poor figures for December.  Clothing
retailers and non-specialist stores were the worst hit with only
internet retailers showing any significant growth, according to the
ONS.

The last time retailers endured a tougher Christmas was 23 years
previously, when sales plunged 1.7%.

The ONS echoed an earlier
caution from Bank of England governor Mervyn King not to read too much
into the poor December figures.  Some analysts put a positive gloss on
the figures, pointing out that the non-seasonally-adjusted figures
showed a performance comparable with 2003. The Novembe

In [9]:
sents = nltk.sent_tokenize(doc.iloc[0].split('\n',1)[1])

In [16]:
featurizer = TfidfVectorizer(stop_words = stopwords.words('english'))

In [17]:
X = featurizer.fit_transform(sents)

In [18]:
def get_sentence_score(tfidf_row):
    x = tfidf_row[tfidf_row!=0]
    return x.mean()

In [19]:
scores = np.zeros(len(sents))
for i in range(len(sents)):
    score = get_sentence_score(X[i,:])
    scores[i] = score

In [20]:
sort_idx = np.argsort(-scores)

It is not neccessary to sort because there are some approaches that need no sorting. 
Also there are many options to choose which sentence should be added to summary. I am using one but there are more than a handful of them

In [21]:
print("Generated Summary: ")
for i in sort_idx[:5]:
    print(wrap("%.2f: %s"%(scores[i], sents[i])))

Generated Summary: 
0.37: A number of retailers have already reported poor figures for
December.
0.35: The ONS revised the annual 2004 rate of growth down from the
5.9% estimated in November to 3.2%.
0.35: However, reports from some High Street retailers highlight the
weakness of the sector.
0.31: "Our view is the Bank of England will keep its powder dry and
wait to see the big picture."
0.31: And a British Retail Consortium survey found that Christmas 2004
was the worst for 10 years.


In [22]:
doc.iloc[0].split("\n",1)[0]

'Christmas sales worst since 1981'